In [48]:
##Packages
from sportsreference.ncaab.schedule import Schedule
from sportsreference.ncaab.teams import Teams
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas
import numpy
pandas.set_option('display.max_columns', 500)
pandas.set_option('display.max_rows', 500)
data = pandas.read_csv('ncaab_box_scores.csv')
data = data[data.columns[-47:]]
FIELDS_TO_DROP = ['team','unique_game','opponent','won','game_date','team_pts', 'opp_pts','team_assist_pct',
'team_block_pct','team_def_reb_pct','team_ft_rate','team_ft_pct','team_off_reb_pct','team_rank','team_stl_pct','team_3pt_rate',
'team_to_pct','team_2pt_rate','opp_assist_pct','opp_block_pct','opp_def_reb_pct','opp_ft_rate','opp_ft_pct','opp_off_reb_pct',
'opp_rank','opp_stl_pct','opp_3pt_rate','opp_to_pct','opp_2pt_rate','team_fg_pct','opp_fg_pct','team_reb_pct',
'opp_reb_pct','team_3pt_pct','team_2pt_pct','opp_3pt_pct','opp_2pt_pct']
data = pandas.get_dummies(data,prefix=['team_home_away'], columns=['team_home_away'])

In [49]:
X = data.drop(FIELDS_TO_DROP, 1).dropna().drop_duplicates()
y = data[['team_pts', 'opp_pts']].values
X_train, X_test, y_train, y_test = train_test_split(X, y)

parameters = {'bootstrap': False,
              'min_samples_leaf': 3,
              'n_estimators': 50,
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 6}
model = RandomForestRegressor(**parameters)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
errors = abs(predictions - y_test)
print('Mean Absolute Error:', round(numpy.mean(errors), 2), 'points.')


Mean Absolute Error: 3.24 points.


In [50]:
importances = list(model.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(X, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: opp_def_rating       Importance: 0.2
Variable: team_off_rating      Importance: 0.19
Variable: opp_off_rating       Importance: 0.18
Variable: team_def_rating      Importance: 0.15
Variable: game_pace            Importance: 0.1
Variable: opp_true_shoot_pct   Importance: 0.07
Variable: team_true_shoot_pct  Importance: 0.04
Variable: team_eff_fg_pct      Importance: 0.03
Variable: opp_eff_fg_pct       Importance: 0.03
Variable: team_home_away_Away  Importance: 0.0
Variable: team_home_away_Home  Importance: 0.0


In [51]:
data2 = data
data2['pred_team_pts'] = pandas.DataFrame(predictions)[0]
data2['pred_opp_pts'] = pandas.DataFrame(predictions)[1]

accuracy = abs(data2['pred_team_pts'] - data2['team_pts'])
accuracy.dropna().mean()

15.097909628915367